In [2]:
import pandas as pd
import numpy as np
import re
import json
import html
# import emot
import string

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import words
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('words')
# nltk.download('wordnet')
nltk.download('stopwords')
tkn = TweetTokenizer()
exclude = set(string.punctuation)
StopWords = stopwords.words("english")
bag = words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
path_to_dataset = "C:/xampp/htdocs/Data Mining/FP/dataset/"
path_to_clean = "C:/xampp/htdocs/Data Mining/FP/clean/"
# filename = "SS-Twitter"
filename = "STS-Gold"
# filename = "STS-Test"

path_to_experiment = path_to_clean + filename +"_EXPERIMENT.csv"

In [81]:
# #Loading dataset 
# Labels = ["class", "id", "date", "query", "username","tweet"] #STS
# Labels = ["id","polarity","tweet"] #Gold
# Labels = ["mean pos","mean neg","tweet"] #SS

# data = pd.read_csv(path_to_dataset + filename +".csv",names=Labels,sep=";",header=0) #Gold
# data = pd.read_csv(path_to_dataset + filename +".csv",names=Labels) #STS
# data = pd.read_csv(path_to_dataset + filename+".csv") #SS
data = pd.read_csv(path_to_experiment) #EXPERIMENT


# data = data[["id", "date", "query", "username","tweet","class"]] #STS-Test
# data = data[['id', 'tweet', 'polarity']] #STS-Gold

data.head(10)

,id,tweet,polarity,NO URL,NO STOPWORD,NO NUMBER,NO REPEAT,NO ACRONYM,NO NEGATION
0,1467933112,the angel is going to miss the athlete this we...,0,angel going miss athlete weekend,the angel is going to miss the athlete this we...,angel going miss athlete weekend,angel going miss athlete weekend,angel going miss athlete weekend,angel going miss athlete weekend
1,2323395086,It looks as though Shaq is getting traded to C...,0,loks though shaq getting traded cleveland play...,it loks as though shaq is getting traded to cl...,loks though shaq getting traded cleveland play...,looks though shaq getting traded cleveland pla...,loks though shaq getting traded cleveland play...,loks though shaq getting traded cleveland play...
2,1467968979,@clarianne APRIL 9TH ISN'T COMING SOON ENOUGH,0,@clariane april th coming soon enough,@clariane april th isn't coming soon enough,@clariane april ninth coming soon enough,@clarianne april th coming soon enough,@clariane april th coming soon enough,@clariane april th coming soon enough
3,1990283756,drinking a McDonalds coffee and not understand...,0,drinking mcdonalds coffee understanding someon...,drinking a mcdonalds coffee and not understand...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...
4,1988884918,So dissapointed Taylor Swift doesnt have a Twi...,0,disapointed taylor swift doesnt twitter,so disapointed taylor swift doesnt have a twitter,disapointed taylor swift doesnt twitter,dissapointed taylor swift doesnt twitter,disapointed taylor swift doesnt twitter,disapointed taylor swift doesnt twitter
5,1467984364,Wishes I was on the Spring Fling Tour with Daw...,0,wishes spring fling tour dawn nece sigh g'knight,wishes i was on the spring fling tour with daw...,wishes spring fling tour dawn nece sigh g'knight,wishes spring fling tour dawn neecee sigh g'kn...,wishes spring fling tour dawn nece sigh g'knight,wishes spring fling tour dawn nece sigh g'knight
6,1964951623,"got a sniffle, got the kids and hubby just lef...",0,got sniffle got kids hubby left work sydney we...,got a sniffle got the kids and hubby just left...,got sniffle got kids hubby left work sydney we...,got sniffle got kids hubby left work sydney we...,got sniffle got kids hubby left work sydney we...,got sniffle got kids hubby left work sydney we...
7,1881133744,i've only been in sydney for 3 hrs but I miss ...,0,i've sydney hours miss friends especially @ktjade,i've only been in sydney for hours but i miss ...,i've sydney 3 hours miss friends especially @k...,i've sydney hours miss friends especially @ktjade,i've sydney hrs miss friends especially @ktjade,i've sydney hours miss friends especially @ktjade
8,1753802024,xboxtweet not working again,0,xboxtwet working,xboxtwet not working again,xboxtwet working,xboxtweet working,xboxtwet working,xboxtwet working
9,1980497384,R.I.P to lebron/kobe puppet commercials...,0,p lebron kobe puppet comercials ...,are i p to lebron kobe puppet comercials ...,p lebron kobe puppet comercials ...,p lebron kobe puppet commercials ...,r p lebron kobe puppet comercials ...,p lebron kobe puppet comercials ...


In [4]:
## CEK Bentuk data

# Cek missing value

print('\nNumber of missing values:')
for col in data.columns:
    print('\t%s: %d' % (col, data[col].isnull().sum()))
    
# cek jumlah perkelas
print('\nNumber of class values:')
print(data.iloc[:,-1].value_counts(dropna=False))


Number of missing values:
	id: 0
	tweet: 0
	polarity: 0

Number of class values:
0    1402
4     632
Name: polarity, dtype: int64


In [3]:
#### PREPROCESSING

## Tokenizing
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    return word_tokens

# #Repeated Word
def normalize_repeated_char(token):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match = r'\1\2\3'
    def replace(old_word):
        if old_word in bag:
#         ps = PorterStemmer()
#         ps.stem(old_word)
#         if wordnet.synsets(old_word):
#         if old_word in words.words():
            return old_word
        new_word = repeat_pattern.sub(match, old_word)
        return replace(new_word) if new_word != old_word else new_word
    
    correct = [replace(word) for word in token]
    return correct

# #Negation
def normalized_negation_phrase(tokenized_sentence):

    mydict = { "did not": ["didn't", "didnt"], 
              "do not": ["don't","dont"], 
              "can not": ["can't","cant"], 
              "sould not": ["souldn't","souldnt"],
              "would not": ["wouldn't","wouldnt"], 
              "could not": ["couldn't","couldnt"],
              "have not": ["haven't","havent"],
              "had not": ["hadn't","hadnt"],
             }
    
    for index in range(len(tokenized_sentence)):
        for key, value in mydict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

# #Stopword
# def stopword(tokenized_sentence):
# #     StopWords = stopwords.words("english")
#     StopWords = stopwords.words("english")
# #     token = nltk.word_tokenize(contoh)
#     for v in tokenized_sentence:
#         if v not in StopWords:
#             return ' '.join(tokenized_sentence)
# #             print(v)

    

# #Akronim
def normalize_slang_words(tokenized_sentence):
    slang_word_dict = json.loads(open("slang_word_dict.txt", 'r').read())
    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

# #Emoticon
def emoticon(tokenized_sentence):
    emoticon_dict = json.loads(open("emoticon_dict.txt", 'r').read())
    for index in range(len(tokenized_sentence)):
        for key, value in emoticon_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue           
    return tokenized_sentence

# Join
# def join_token(tokenized_sentence):
#     for index in range(len(tokenized_sentence)):
#         tokenized_sentence[index] = '|'.join(str(i) for i in tokenized_sentence[index])
#     return ' '.join(tokenized_sentence)

In [82]:
def preprocessing(comment):
    
    comment = comment.lower() #lower
    comment = html.unescape(comment) #html tag
    comment = re.sub("@[A-Za-z0-9]+", "", comment) #no Mention
    comment = re.sub("#[A-Za-z0-9]+", "", comment) #no Hashtag
    comment = re.sub("[0-9]", "", comment) #no Number
    comment = re.sub(r"http\S+", "", comment) #no HTML    
    comment = tkn.tokenize(comment)#token
    comment = emoticon(comment) #Emoticon
    comment = normalize_repeated_char(comment) #Repeated
    comment = normalized_negation_phrase(comment) #Negation
    comment = normalize_slang_words(comment) #slang
    comment = [word for word in comment if word not in StopWords] #Stopword
    comment = ' '.join(ch for ch in comment if ch not in exclude) #Puntuation
    
    return comment

In [83]:
data['OUR'] = data['tweet'].apply(preprocessing)
data.head()
# data['Clean'].loc[4]

,id,tweet,polarity,NO URL,NO STOPWORD,NO NUMBER,NO REPEAT,NO ACRONYM,NO NEGATION,OUR
0,1467933112,the angel is going to miss the athlete this we...,0,angel going miss athlete weekend,the angel is going to miss the athlete this we...,angel going miss athlete weekend,angel going miss athlete weekend,angel going miss athlete weekend,angel going miss athlete weekend,angel going miss athlete weekend
1,2323395086,It looks as though Shaq is getting traded to C...,0,loks though shaq getting traded cleveland play...,it loks as though shaq is getting traded to cl...,loks though shaq getting traded cleveland play...,looks though shaq getting traded cleveland pla...,loks though shaq getting traded cleveland play...,loks though shaq getting traded cleveland play...,loks though shaq getting traded cleveland play...
2,1467968979,@clarianne APRIL 9TH ISN'T COMING SOON ENOUGH,0,@clariane april th coming soon enough,@clariane april th isn't coming soon enough,@clariane april ninth coming soon enough,@clarianne april th coming soon enough,@clariane april th coming soon enough,@clariane april th coming soon enough,april th coming soon enough
3,1990283756,drinking a McDonalds coffee and not understand...,0,drinking mcdonalds coffee understanding someon...,drinking a mcdonalds coffee and not understand...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...,drinking mcdonalds coffee understanding someon...
4,1988884918,So dissapointed Taylor Swift doesnt have a Twi...,0,disapointed taylor swift doesnt twitter,so disapointed taylor swift doesnt have a twitter,disapointed taylor swift doesnt twitter,dissapointed taylor swift doesnt twitter,disapointed taylor swift doesnt twitter,disapointed taylor swift doesnt twitter,disapointed taylor swift doesnt twitter


In [84]:
## Save Clean Data

# data.to_csv(path_to_clean+filename+"_BASELINE.csv",index=False) 
# #with url, stopword, number, repeated, acronym, negation

data.to_csv(path_to_clean+filename+"_EXPERIMENT.csv",index=False) 
# #combination of url, stopword, number, repeated, acronym, negation, our preprocessing

In [69]:
# # #Make Dict

# # #Join
# file1 = pd.read_csv(path_to_clean + "STS-Test.csv")
# file2 = pd.read_csv(path_to_clean + "SS-Twitter.csv")
# file3 = pd.read_csv(path_to_clean + "STS-Gold.csv")

# s1 = file1['Clean']
# s2 = file2['Clean']
# s3 = file3['Clean']

# tweet = list()
# tweet = s1.append(s2)
# tweet = tweet.append(s3)
# # tweet.reset_index()


In [126]:
# # #Save
# tweet.to_csv(path_to_clean+"All_Tweet.csv",index=False)

In [127]:
# all_tweet = pd.read_csv(path_to_clean + "All_Tweet.csv",header=None)
# all_tweet.head()

In [128]:
# # # Emoticon Dict

# for index, row in all_tweet.iterrows():
#     ContentnoMention = re.sub("[A-Za-z]","",str(row))

#     print(ContentnoMention)

In [155]:
# # #Check kata baku
# def is_english_word(comment):
#     # creation of this dictionary would be done outside of 
#     #     the function because you only need to do it once.
#     comment = comment.lower() #lower
#     comment = html.unescape(comment) #html tag
#     comment = re.sub("@[A-Za-z0-9]+", "", comment) #no Mention
#     comment = re.sub("#[A-Za-z0-9]+", "", comment) #no Hashtag
#     comment = re.sub("[0-9]", "", comment) #no Number
#     comment = re.sub(r"http\S+", "", comment) #no HTML    
#     comment = tkn.tokenize(comment)#token
#     dictionary = dict.fromkeys(words.words(), None)
#     for i,v in enumerate(comment):
#         try:
#             x = dictionary[v]
#             continue
#         except KeyError:
#             print (v)

In [154]:
# data['Tweet'].apply(is_english_word)

In [112]:
# # Scrapping

# from bs4 import BeautifulSoup
# import requests, json
# resp = requests.get('http://www.netlingo.com/acronyms.php')
# soup = BeautifulSoup(resp.text, "html.parser")
# slangdict= {}
# key=""
# value=""
# for div in soup.findAll('div', attrs={'class':'list_box3'}):
#     for li in div.findAll('li'):
#         for a in li.findAll('a'):
#             key =a.text
#         value = li.text.split(key)[1]
#         slangdict[key]=value
# with open('C:/Users/USER/Downloads/tegar nitip/SMT 8/DatMin/FP/myslang.json', 'w') as find:
#     json.dump(slangdict,find,indent=2)

In [8]:
# Convert to csv

# data = pd.read_csv(path_to_dataset + filename, delimiter="\t", encoding='latin1')
# data.to_csv(path_to_dataset+"SS-Twitter.csv",index=False)
# # f = open(path_to_dataset + filename, "r")
# # content = f.read()